In [ ]:
# DGHK23，嵌入不等式hint， <v,s> <= l
cd framework

In [ ]:
load("../framework/instance_gen.sage")
import numpy as np
import random

In [ ]:
n = 80
m = n
q = 3329
D_s = build_centered_binomial_law(40)
D_e = D_s
A, b, s, dbdd = initialize_from_LWE_instance(DBDD, n, q, m, D_e, D_s)
es = np.array(dbdd.u[0][:-1]) 
print(es)
beta, delta = dbdd.estimate_attack()

In [ ]:
# 使用拒绝采样生成[-3*sigma,3*sigma]之间满足高斯分布的随机数
def sample_discrete_gaussian(sigma):
    while True:
        x = random.randint(int(-3 * sigma), int(3 * sigma)) 
        acceptance_prob = math.exp(-x**2 / (2 * sigma**2)) 
        if random.uniform(0, 1) < acceptance_prob: 
            return x
            

def generate_se_eta_perfect_hint(m, n, q, k):
  V = []
  L = []

  for i in range(k):
    D_s = build_centered_binomial_law(3)
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    l = dbdd.leak(v)
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L

        
def generate_se_eta_approx_hint(m, n, q, sigma, k):
  V = []
  L = []

  for i in range(k):
    D_e = {-3: 1/64, -2: 6/64, -1: 15/64, 0: 20/64, 1: 15/64, 2: 6/64, 3: 1/64}
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    e = sample_discrete_gaussian(sigma)
    l = dbdd.leak(v) + e
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L


# 生成不等式hint，<v,s>=l-noisy，<v,s><l
def generate_se_eta_sigma_ineq_hint(m, n, q, sigma, k):
  V = []
  L = []

  for i in range(k):
    D_e = {-3: 1/64, -2: 6/64, -1: 15/64, 0: 20/64, 1: 15/64, 2: 6/64, 3: 1/64}
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    v_vec = vec(v)
    noisy = sample_discrete_gaussian(sigma)+ 3 * sigma
    # print(noisy)
    l = ebdd.leak(v_vec) + noisy
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L

In [ ]:
# perfect hint 
Num = []
Dis = []
num_hint = 10
V, L = generate_se_eta_perfect_hint(m, n, q, num_hint)
for j in range(num_hint):
    if j % 1 == 0:
        Num.append(j)
        secret = dbdd.attack(beta_max=32)
        guess = np.array(secret[1][0])[:-1]
        short_vector = np.array(es - guess)
        distance = np.linalg.norm(short_vector)
        print("distance",distance)
        Dis.append(distance)
        if distance == 0:
            break
    print("the ",j+1,"-th secret error perfect hint")
    _ = dbdd.integrate_perfect_hint(vec(V[j]), L[j])
print(Num)
print(Dis)

In [ ]:
# approimate hint 
Num = []
Dis = []
num_hint = 30
sigma = 2
V, L = generate_se_eta_approx_hint(m, n, q, sigma, num_hint)
for j in range(num_hint):
    if j % 2 == 0:
        Num.append(j)
        secret = dbdd.attack(beta_max=32)
        guess = np.array(secret[1][0])[:-1]
        short_vector = np.array(es - guess)
        distance = np.linalg.norm(short_vector)
        print("distance",distance)
        Dis.append(distance)
        if distance == 0:
            break
    print("the ",j+1,"-th secret error approx hint")
    _ = dbdd.integrate_approx_hint(vec(V[j]), L[j], sigma**2, aposteriori=False)
print(Num)
print(Dis)

In [ ]:
# ineq hint 
Num = []
Dis = []
num_hint = 500
sigma = 2
V, L = generate_se_eta_sigma_ineq_hint(m, n, q, sigma, num_hint)
for j in range(num_hint):
    if j % 40 == 0:
        Num.append(j)
        secret = ebdd.attack(beta_max=32)
        guess = np.array(secret[1][0])[:-1]
        short_vector = np.array(es - guess)
        distance = np.linalg.norm(short_vector)
        print("distance",distance)
        Dis.append(distance)
        if distance == 0:
            break
    print("the ",j+1,"-th secret error ineq hint")
    _ = ebdd.integrate_ineq_hint(vec(V[j]), L[j])
print(Num)
print(Dis)

In [ ]:
for j in range(num_hint):
    print("the ",j+1,"-th secret error inequality hint")
    _ = dbdd.integrate_ineq_hint(vec(V[j]), L[j], sigma**2, aposteriori=False)